Download review data from https://www.yelp.com/dataset_challenge/dataset

In [1]:
import graphlab as gl

In [5]:
import pandas as pd
import re
import json

def yelp2df(filename):
    with open(filename, 'r') as f:
        reviews = f.readlines()

    reviews_json=[json.loads(review) for review in reviews]
    df = pd.DataFrame(reviews_json)

    return df

In [9]:
DATA_PATH="/Volumes/Media2/Datasets/yelp_dataset_challenge_round9/Reviews-Split/xaa"

In [10]:
#df_business = yelp2df("/Users/gazelle/xaa")
df_reviews = yelp2df(DATA_PATH)

In [17]:
df_reviews.head()

,business_id,cool,date,funny,review_id,stars,text,type,useful,user_id
0,2aFiy99vNLklCx3T_tGS9A,0,2011-10-10,0,NxL8SIC5yqOdnlXCg18IBg,5,If you enjoy service by someone who is as comp...,review,0,KpkOkG6RIf4Ra25Lhhxf1A
1,2aFiy99vNLklCx3T_tGS9A,0,2010-12-29,0,pXbbIgOXvLuTi_SPs1hQEQ,5,After being on the phone with Verizon Wireless...,review,1,bQ7fQq1otn9hKX-gXRsrgA
2,2aFiy99vNLklCx3T_tGS9A,0,2011-04-29,0,wslW2Lu4NYylb1jEapAGsw,5,Great service! Corey is very service oriented....,review,0,r1NUhdNmL6yU9Bn-Yx6FTw
3,2LfIuF3_sX6uwe-IR-P0jQ,1,2014-07-14,0,GP6YEearUWrzPtQYSF1vVg,5,Highly recommended. Went in yesterday looking ...,review,0,aW3ix1KNZAvoM8q-WghA3Q
4,2LfIuF3_sX6uwe-IR-P0jQ,0,2014-01-15,0,25RlYGq2s5qShi-pn3ufVA,4,I walked in here looking for a specific piece ...,review,0,YOo-Cip8HqvKp_p9nEGphw


In [11]:
gl_sframe = gl.SFrame(df_reviews)

In [16]:
model_rec = gl.recommender.item_similarity_recommender.create(gl_sframe, 
                                                        user_id='user_id',
                                                        item_id='business_id',
                                                        target='stars',
                                                        user_data=None,
                                                        item_data=None,
                                                        nearest_items=None,
                                                        similarity_type='pearson',
                                                        threshold=0.001,
                                                        only_top_k=64,
                                                        verbose=True,
                                                        target_memory_usage=8589934592)

Recsys training: model = item_similarity

Warning: Ignoring columns cool, date, funny, review_id, text, type, useful;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 200000 observations with 116127 users and 25901 items.

...

| 35.64s                              | 100              | 25901           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 35.8155s

In [15]:
folds = gl.cross_validation.KFold(gl_sframe, 5)
params = dict(user_id='user_id',
                                                        item_id='business_id',
                                                        target='stars',
                                                        user_data=None,
                                                        item_data=None,
                                                        nearest_items=None,
                                                        similarity_type='pearson',
                                                        threshold=0.001,
                                                        only_top_k=64,
                                                        verbose=True,
                                                        target_memory_usage=8589934592)
job = gl.cross_validation.cross_val_score(folds,
                                              gl.recommender.item_similarity_recommender.create,
                                              params)
print job.get_results()

[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.job: Creating a LocalAsync environment called 'async'.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Cross-Validation-Apr-23-2017-12-26-07-030824--6299041' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Cross-Validation-Apr-23-2017-12-26-07-030824--6299041' scheduled.
[INFO] graphlab.deploy._job: Waiting for job to finish, this may take quite a while.
[INFO] graphlab.deploy._job: You may CTRL-C to stop this command and it will not cancel your job.


OrderedDict([('models', [Class                            : ItemSimilarityRecommender

Schema
------
User ID                          : user_id
Item ID                          : business_id
Target                           : stars
Additional observation features  : 0
User side features               : []
Item side features               : []

Statistics
----------
Number of observations           : 160000
Number of users                  : 98387
Number of items                  : 20205

Training summary
----------------
Training time                    : 22.592

Model Parameters
----------------
Model class                      : ItemSimilarityRecommender
threshold                        : 0.001
similarity_type                  : pearson
training_method                  : auto

Other Settings
--------------
degree_approximation_threshold   : 4096
sparse_density_estimation_sample_size : 4096
max_data_passes                  : 4096
target_memory_usage              : 8589934592
seed_item

In [20]:
model_rec.recommend(users=None,k=1)

recommendations finished on 1000/116127 queries. users per second: 1724.01

recommendations finished on 2000/116127 queries. users per second: 1918.27

recommendations finished on 3000/116127 queries. users per second: 1856.8

recommendations finished on 4000/116127 queries. users per second: 1911.32

recommendations finished on 5000/116127 queries. users per second: 1995.74

...

recommendations finished on 113000/116127 queries. users per second: 1869.6

recommendations finished on 114000/116127 queries. users per second: 1859.52

recommendations finished on 115000/116127 queries. users per second: 1850.81

recommendations finished on 116000/116127 queries. users per second: 1845.23

user_id,business_id,score,rank
KpkOkG6RIf4Ra25Lhhxf1A,78iZO7fwyQ_Bhq63p4uXxA,5.0,1
bQ7fQq1otn9hKX-gXRsrgA,78iZO7fwyQ_Bhq63p4uXxA,5.0,1
r1NUhdNmL6yU9Bn-Yx6FTw,78iZO7fwyQ_Bhq63p4uXxA,5.0,1
aW3ix1KNZAvoM8q-WghA3Q,2aFiy99vNLklCx3T_tGS9A,5.0,1
YOo-Cip8HqvKp_p9nEGphw,2aFiy99vNLklCx3T_tGS9A,5.0,1
bgl3j8yJcRO-00NkUYsXGQ,2aFiy99vNLklCx3T_tGS9A,5.0,1
CWKF9de-nskLYEqDDCfubg,2aFiy99vNLklCx3T_tGS9A,5.0,1
GJ7PTY7huYORFKKg3db3Gw,2aFiy99vNLklCx3T_tGS9A,5.0,1
rxqp9eXZj1jYTn0UIsm3Hg,2aFiy99vNLklCx3T_tGS9A,5.0,1
UU0nHQtHPMAfLidk8tOHTg,2aFiy99vNLklCx3T_tGS9A,5.0,1


In [23]:
model_rec.recommend(users=gl_sframe['user_id'][0:5],k=8)

user_id,business_id,score,rank
KpkOkG6RIf4Ra25Lhhxf1A,2iUGRHPUApKKJM8CBGqmNw,5.0,1
KpkOkG6RIf4Ra25Lhhxf1A,A-HeICDZo79ty0OaUSAJVw,5.0,2
KpkOkG6RIf4Ra25Lhhxf1A,2YcK8gUaY2VAvHUGE3NakQ,5.0,3
KpkOkG6RIf4Ra25Lhhxf1A,6Ivnwybl4sTvcwMtkkjrQQ,5.0,4
KpkOkG6RIf4Ra25Lhhxf1A,44d0NDLZiLTGmpBlNdhDaw,5.0,5
KpkOkG6RIf4Ra25Lhhxf1A,7vXXRslK05EyPoRVMqtzdA,5.0,6
KpkOkG6RIf4Ra25Lhhxf1A,0vytvfym7q8f85jeOvgr5Q,5.0,7
KpkOkG6RIf4Ra25Lhhxf1A,78iZO7fwyQ_Bhq63p4uXxA,5.0,8
bQ7fQq1otn9hKX-gXRsrgA,2iUGRHPUApKKJM8CBGqmNw,5.0,1
bQ7fQq1otn9hKX-gXRsrgA,A-HeICDZo79ty0OaUSAJVw,5.0,2
